<a href="https://colab.research.google.com/github/epicskills1/Spam-Mail-Prediction-using-machine-learning/blob/main/Spam_email_messages_predictionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install streamlit pyngrok

In [14]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from pyngrok import ngrok

# Load and preprocess the data
def load_data():
    raw_mail_data = pd.read_csv('/content/mail_data (1).csv')
    mail_data = raw_mail_data.where(pd.notnull(raw_mail_data), '')  # Replace NaN values with empty string

    # Label encoding: spam = 0, ham = 1
    mail_data.loc[mail_data['Category'] == 'spam', 'Category'] = 0
    mail_data.loc[mail_data['Category'] == 'ham', 'Category'] = 1

    X = mail_data['Message']
    Y = mail_data['Category'].astype('int')  # Convert target to integer

    return X, Y

# Train the model
def train_model(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

    # Text feature extraction using TF-IDF
    feature_extraction = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
    X_train_features = feature_extraction.fit_transform(X_train)
    X_test_features = feature_extraction.transform(X_test)

    # Train Logistic Regression model
    model = LogisticRegression()
    model.fit(X_train_features, Y_train)

    # Accuracy on test data
    predictions_test = model.predict(X_test_features)
    accuracy_test = accuracy_score(Y_test, predictions_test)

    return model, feature_extraction, accuracy_test

# Make predictions
def predict_mail(model, feature_extraction, mail):
    input_data_features = feature_extraction.transform([mail])
    prediction = model.predict(input_data_features)

    if prediction[0] == 1:
        return 'Ham mail'
    else:
        return 'Spam mail'

# Streamlit app
def main():
    st.title('Spam Mail Detection App')

    st.write("Enter the email message below:")

    # User input for the email message
    user_input = st.text_area('Email content', height=200)

    # Load and train the model
    X, Y = load_data()
    model, feature_extraction, accuracy_test = train_model(X, Y)

    # Display accuracy
    st.write(f"Model Accuracy on test data: {accuracy_test:.2%}")

    # Prediction when the user clicks the button
    if st.button('Predict'):
        if user_input:
            result = predict_mail(model, feature_extraction, user_input)
            st.write(f"Prediction: **{result}**")
        else:
             st.write("Please enter an email message to classify.")

if __name__ == '__main__':
    main()


Overwriting app.py


In [15]:
!pip install --upgrade pyngrok

from pyngrok import ngrok
ngrok.set_auth_token("2mwNt6HZ5mS0Nelnocs2W34Pjw8_5WUFreV5Y7kxWcFp3AXuJ") # Removed extra space before this line

# Terminate all existing ngrok tunnels
ngrok.kill()

# Create a new ngrok tunnel, explicitly specifying HTTP protocol
public_url = ngrok.connect(8501, proto="http") # Specify protocol as "http"
print(f"Streamlit App URL: {public_url}")


Streamlit App URL: NgrokTunnel: "https://eb2c-35-245-108-212.ngrok-free.app" -> "http://localhost:8501"


In [16]:
!pip install streamlit
import subprocess

# Run Streamlit app
subprocess.Popen(['streamlit', 'run', 'app.py'])

<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>